In [2]:
val rawTickDfDf = spark.read.format("csv")
                .option("header",true)
                .option("inferSchema",true)
                .load("hdfs://localhost:9000/layers/raw/csv/")
rawTickDfDf.printSchema()
rawTickDfDf.show(5)

root
 |-- symbol: string (nullable = true)
 |-- volume: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- MIN: integer (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- HOUR: integer (nullable = true)
 |-- SYMBOL1: string (nullable = true)

+------+------+-----+--------------------+---+----+-----+---+----+-------+
|symbol|volume|price|                time|MIN|YEAR|MONTH|DAY|HOUR|SYMBOL1|
+------+------+-----+--------------------+---+----+-----+---+----+-------+
|MARUTI|    40| 30.0|2022-03-20 23:00:...|  0|2022|    3| 20|  23| MARUTI|
|MARUTI|   120| 50.0|2022-03-20 23:01:...|  1|2022|    3| 20|  23| MARUTI|
|MARUTI|   120|  6.0|2022-03-20 23:01:...|  1|2022|    3| 20|  23| MARUTI|
|MARUTI|   120| 43.0|2022-03-20 23:01:...|  1|2022|    3| 20|  23| MARUTI|
|MARUTI|   120| 26.0|2022-03-20 23:01:...|  1|2022|    3| 20|  23| MARUTI|
+------+------+---

rawTickDfDf: org.apache.spark.sql.DataFrame = [symbol: string, volume: int ... 8 more fields]


In [5]:

import org.apache.spark.sql.functions._
//.filter( (col("total_ratings") >= 100) & (col("avg_rating") >=3.5) )\
val filterDf = rawTickDfDf.filter((col("price") > 0.0 )&&(col( "volume" )> 0)).drop("HOUR","MIN")
//#filterDf = rawTickDfDf.filter("(volume > 0)")
filterDf.show(5)

+------+------+-----+--------------------+----+-----+---+-------+
|symbol|volume|price|                time|YEAR|MONTH|DAY|SYMBOL1|
+------+------+-----+--------------------+----+-----+---+-------+
|MARUTI|    40| 30.0|2022-03-20 23:00:...|2022|    3| 20| MARUTI|
|MARUTI|   120| 50.0|2022-03-20 23:01:...|2022|    3| 20| MARUTI|
|MARUTI|   120|  6.0|2022-03-20 23:01:...|2022|    3| 20| MARUTI|
|MARUTI|   120| 43.0|2022-03-20 23:01:...|2022|    3| 20| MARUTI|
|MARUTI|   120| 26.0|2022-03-20 23:01:...|2022|    3| 20| MARUTI|
+------+------+-----+--------------------+----+-----+---+-------+
only showing top 5 rows



import org.apache.spark.sql.functions._
filterDf: org.apache.spark.sql.DataFrame = [symbol: string, volume: int ... 6 more fields]


In [7]:
filterDf.write.partitionBy("YEAR", "MONTH", "DAY","SYMBOL1").mode("overwrite").option("header", true)
                .parquet("hdfs://localhost:9000/layers/raw/parquet1Scala")